In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm_notebook as tqdm
from flask import jsonify

# Here the constants
model_path = "./mnist_torch.pt"
device = "cpu"
batch_size_train = 64
batch_size_test = 1024
image_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# All functions below are for loading and train models, no any code outside functions so we can safely transform this notebook to python file >>>>>>>>>>>
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5, stride=1)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.conv2_drop(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        x = x.view(-1, 320)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x)


def load_model(model_path):
    #model = CNN()
    #model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    model = torch.load(model_path, map_location=torch.device(device))
    model.eval()  # Set the model to evaluation mode
    return model

def preprocess_image(image_path):
    image = Image.open(image_path)
    image = image_transform(image).unsqueeze(0)
    return image

def data_loader():
    train_dataset = torchvision.datasets.MNIST('dataset/', train=True, download=True, transform=image_transform)
    test_dataset = torchvision.datasets.MNIST('dataset/', train=False, download=True, transform=image_transform)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size_test, shuffle=True)
    return train_dataset,train_loader,test_dataset,test_loader


def train(model, device, train_loader, optimizer, num_epochs, log_interval=10000):
    model.train()
    for epoch in range(1, num_epochs + 1):
        tk0 = tqdm(train_loader, total=len(train_loader))
        counter = 0
        for batch_idx, (data, target) in enumerate(tk0):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            counter += 1
            tk0.set_postfix(loss=(loss.item() * data.size(0) / (counter * train_loader.batch_size)))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
# <<<<<<<<<<<<<<<< All functions above are for loading and train models, no any code outside functions so we can safely transform this notebook to python file

# >>>>>>>>> All functions below to run training or prediction. Uncomment function in the bottom to train model or test prediction before deployment
#run this function to train model
def train_model():
    model = CNN()
    num_epochs = 3
    learning_rate = 0.01
    momentum = 0.5
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
    _,train_loader,_,test_loader = data_loader()
    train(model, device, train_loader, optimizer, num_epochs)
    test(model, device, test_loader)
    torch.save(model, model_path)
    

# THIS IS THE MAIN FUNCTION FOR API: to predict image class, we will use it in API setup
def predict_image(image_path):
    image = preprocess_image(image_path)
    with torch.no_grad():
        model = load_model(model_path)
        output = model(image)
        predicted_class = torch.argmax(output, dim=1).item()
    #we use Flask to deploy so we should return Flask Response, we use jsonify
    return jsonify(predicted_class)

#train
#comment for api deploy, uncomment for training
#train_model()

# Predict the class of a new image
#comment for api deploy, uncomment for prediction when you test notebook before deployment
#predicted_class = predict_image("mnist-5.png")
#print("Predicted class:", predicted_class)



/tmp/ipykernel_82/1778218597.py:69: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tk0 = tqdm(train_loader, total=len(train_loader))


  1%|█                   | 3/938 [00:00<00:34, 27.31it/s, loss=0.456]

/tmp/ipykernel_82/1778218597.py:43: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


  100%|████████████████████| 938/938 [00:42<00:00, 18.40it/s, loss=0.000113]

  100%|████████████████████| 938/938 [00:43<00:00, 17.94it/s, loss=0.000111]


Test set: Average loss: 0.1878, Accuracy: 9452/10000 (95%)

Predicted class: 5
